In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as sklearn_pca
from locale import atof
from sklearn import decomposition
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import ensemble
from sklearn.naive_bayes import BernoulliNB
import json

In [2]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [3]:
#Import data from json file and create a list

data = []
with open('Digital_Music_5.json') as f:
    for line in f:
        data.append(json.loads(line))

#Create a dataframe with the columns that are interesting for this exercise
#Columns left out: 'helpful', 'reviewTime', 'reviewerID','reviewerName'
names = ["overall",'summary', "reviewText"]
amazonraw = pd.DataFrame(data, columns=names)
amazonraw.head()

,overall,summary,reviewText
0,5.0,Enya's last great album,"It's hard to believe ""Memory of Trees"" came ou..."
1,5.0,Enya at her most elegant,"A clasically-styled and introverted album, Mem..."
2,5.0,The best so far,I never thought Enya would reach the sublime h...
3,5.0,Ireland produces good music.,This is the third review of an irish album I w...
4,4.0,4.5; music to dream to,"Enya, despite being a successful recording art..."


In [4]:
#Analyse the dataset: types, length of the dataframe and NaN
amazonraw.info()
print('length of the dataframe:', len(amazonraw))
print('Nans in the dataframe:', amazonraw.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64706 entries, 0 to 64705
Data columns (total 3 columns):
overall       64706 non-null float64
summary       64706 non-null object
reviewText    64706 non-null object
dtypes: float64(1), object(2)
memory usage: 1.5+ MB
length of the dataframe: 64706
Nans in the dataframe: overall       0
summary       0
reviewText    0
dtype: int64


In [5]:
amazonraw.overall.unique()

array([ 5.,  4.,  3.,  1.,  2.])

In [6]:
#Change the Overall variable into a categorical variable
amazonraw.loc[amazonraw['overall'] == 3, 'Sentiment'] = 0
amazonraw.loc[amazonraw['overall'] <= 2, 'Sentiment'] = -1
amazonraw.loc[amazonraw['overall'] >=4 , 'Sentiment'] = 1
amazonraw.loc[amazonraw['Sentiment'] == -1, 'Category'] ='Negative'
amazonraw.loc[amazonraw['Sentiment'] == 1, 'Category'] = 'Positive'
amazonraw.loc[amazonraw['Sentiment'] == 0, 'Category'] = 'Neutral'

In [7]:
names = ['Category',"overall",'Sentiment','summary', "reviewText"]
amazon = pd.DataFrame(amazonraw, columns=names)
amazon.head()

,Category,overall,Sentiment,summary,reviewText
0,Positive,5.0,1.0,Enya's last great album,"It's hard to believe ""Memory of Trees"" came ou..."
1,Positive,5.0,1.0,Enya at her most elegant,"A clasically-styled and introverted album, Mem..."
2,Positive,5.0,1.0,The best so far,I never thought Enya would reach the sublime h...
3,Positive,5.0,1.0,Ireland produces good music.,This is the third review of an irish album I w...
4,Positive,4.0,1.0,4.5; music to dream to,"Enya, despite being a successful recording art..."


In [8]:
amazon.Category.value_counts()

Positive    52116
Neutral      6789
Negative     5801
Name: Category, dtype: int64

In [20]:
#Downsample majority class (due to computational restrictions we downsample the majority instead of upsampling the minority)

# Separate majority and minority classes
amazon_majority = amazon[amazon.Sentiment == 1]
amazon_minority = amazon[amazon.Sentiment < 1]
 
# Downsample mairlinesass
amazon_majority_downsampled = resample(amazon_majority, replace=False, n_samples=6200, random_state=123) 
 
# Combine minority class with downsampled majority class
amazon1 = pd.concat([amazon_majority_downsampled, amazon_minority])
 
# Display new class counts
amazon1.Category.value_counts()
amazon1.head()

,Category,overall,Sentiment,summary,reviewText
59643,Positive,5.0,1.0,Timeless,This is a great song that will stand the test ...
55386,Positive,5.0,1.0,trite piece of garbage,I am ashamed to say that such a high rating on...
29299,Positive,4.0,1.0,nicely EVERY MOBB FAN SHOULD HAVE IT,"Mobb deep have never disapointed fans , with a..."
61457,Positive,4.0,1.0,"""Like Cigarette Ash The World Is Collapsing Ar...",I first learned of Sharon Van Etten when Rolli...
41719,Positive,4.0,1.0,"4.5 stars: passionate, grinds you down...",In a similar line as contemporary acts The Vin...


In [10]:
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer

In [11]:
# based on http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(reviewText):
    # remove non letters
    reviewText = re.sub("[^a-zA-Z]", " ", reviewText)
    # tokenize
    tokens = nltk.word_tokenize(reviewText)
    # stem
    stems = stem_tokens(tokens, stemmer)
    return stems
######## 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = 'english'
)

In [12]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [14]:
corpus_data_features = vectorizer.fit_transform(
    amazon1.reviewText.tolist())

In [15]:
corpus_data_features_nd = corpus_data_features.toarray()
corpus_data_features_nd.shape

(18790, 40897)

In [16]:
vocab = vectorizer.get_feature_names()
print (vocab)

['aa', 'aaa', 'aaaa', 'aaaaaaahhhh', 'aaaaaaalllllllll', 'aaaaaah', 'aaaaaayyy', 'aaaaall', 'aaaanyway', 'aaaaww', 'aaack', 'aaah', 'aaahh', 'aabb', 'aaberg', 'aacut', 'aad', 'aah', 'aahh', 'aaight', 'aalania', 'aaliyah', 'aalyiah', 'aand', 'aaow', 'aarggg', 'aargh', 'aaron', 'aaronmagram', 'aasim', 'aask', 'aaway', 'aayyyyoooo', 'ab', 'abab', 'abacab', 'aback', 'abacu', 'abad', 'abaddon', 'abandon', 'abandonada', 'abandond', 'abat', 'abb', 'abba', 'abber', 'abbey', 'abbi', 'abbl', 'abboitoir', 'abbott', 'abbrevi', 'abc', 'abcens', 'abchao', 'abd', 'abdic', 'abduct', 'abdul', 'abe', 'abeat', 'abeauti', 'abel', 'abend', 'aber', 'abercrombi', 'aberdeen', 'aberr', 'abesntia', 'abet', 'abhor', 'abhorr', 'abid', 'abig', 'abil', 'abili', 'abilli', 'abilt', 'abism', 'abit', 'abiut', 'abject', 'abk', 'abkco', 'abl', 'ablaz', 'ablbum', 'ableto', 'abley', 'abli', 'ablomb', 'ablum', 'abnorm', 'aboard', 'aboat', 'abod', 'abolish', 'abomin', 'aborb', 'aborigin', 'abort', 'abortiondeep', 'abotu', 'a

In [18]:
# Sum up the counts of each vocabulary word
dist = np.sum(corpus_data_features_nd, axis=0)
    
# For each, print the vocabulary word and the number of times it 
# appears in the data set
for tag, count in zip(vocab, dist):
    print (count, tag)

3 aa
3 aaa
1 aaaa
1 aaaaaaahhhh
1 aaaaaaalllllllll
1 aaaaaah
1 aaaaaayyy
1 aaaaall
1 aaaanyway
1 aaaaww
1 aaack
1 aaah
1 aaahh
1 aabb
2 aaberg
37 aacut
2 aad
2 aah
1 aahh
6 aaight
1 aalania
47 aaliyah
1 aalyiah
1 aand
1 aaow
1 aarggg
1 aargh
57 aaron
1 aaronmagram
1 aasim
1 aask
1 aaway
1 aayyyyoooo
6 ab
1 abab
4 abacab
4 aback
1 abacu
1 abad
3 abaddon
148 abandon
1 abandonada
1 abandond
2 abat
1 abb
43 abba
1 abber
19 abbey
2 abbi
1 abbl
1 abboitoir
5 abbott
7 abbrevi
40 abc
1 abcens
1 abchao
2 abd
2 abdic
1 abduct
22 abdul
4 abe
1 abeat
1 abeauti
20 abel
1 abend
1 aber
4 abercrombi
1 aberdeen
2 aberr
1 abesntia
5 abet
4 abhor
2 abhorr
6 abid
2 abig
385 abil
1 abili
1 abilli
1 abilt
1 abism
12 abit
1 abiut
3 abject
11 abk
3 abkco
642 abl
2 ablaz
1 ablbum
1 ableto
1 abley
11 abli
1 ablomb
26 ablum
4 abnorm
14 aboard
1 aboat
1 abod
1 abolish
29 abomin
1 aborb
2 aborigin
23 abort
1 abortiondeep
1 abotu
1 aboult
19 abound
1 aboutblack
1 abouthold
1 abouthop
1 abouthow
1 abouti
1 aboutin
1

1 anymorea
1 anymorecaus
1 anymoreinsid
1 anymoresanta
1 anyodi
1089 anyon
1 anyonw
4 anyoth
2 anyplac
2 anyt
1792 anyth
3 anythign
2 anythin
65 anytim
1 anyuuu
1 anywayi
12 anywayz
215 anywher
8 anywho
1 anza
1 aobut
9 aol
1 aongst
1 aoout
30 aor
3 aorta
1 aorund
1 aot
1 aoveral
7 apach
1 apagaria
1 apain
1 apair
1 apapt
1 apar
1 aparr
316 apart
1 apartfrom
2 apartheid
1 aparthi
9 apathet
7 apathi
1 apc
25 ape
3 apeal
2 apear
1 apepar
1 aper
1 apethecari
20 apex
8 aphasia
7 aphex
1 aphrodhis
6 aphrodisiac
1 aphrodit
5 apiec
1 apingroxi
1 aplay
9 aplenti
1 apli
2 aplog
4 aplomb
1 apo
1 apocalpys
10 apocalyps
1 apocalypso
12 apocalypt
1 apocrypha
1 apoge
1 apolagi
1 apolig
2 apolit
30 apollo
1 apollonia
66 apolog
5 apologet
3 apologis
2 apologist
1 aponea
1 apoplect
1 aportan
1 apost
2 apostl
2 apothecari
1 apotheosi
40 app
1 appair
15 appal
1 appalachia
4 appalachian
1 appalingli
1 appallinggo
4 appallingli
1 appaloosa
314 appar
4 apparantli
4 apparatu
1 apparit
1 apparr
1 appart
1 app

4 badboy
5 badd
1 baddbonni
2 badder
24 baddest
1 badeveryth
1 badfing
1 badfish
2 badg
2 badger
1 badim
1 badin
1 badiuzm
1 badl
5 badland
102 badli
1 badlyr
1 badman
11 badmotorfing
2 badn
1 badnoth
1 badnothingth
1 badnow
1 badonkadonk
2 badth
1 badto
115 badu
3 baduism
19 baduizm
1 badunkadunk
1 badvirtu
3 badway
19 baez
29 baffl
1 bafflingli
1 bafter
186 bag
18 baggag
2 bagger
1 baggeri
4 baggi
1 baggin
6 baghdad
1 bagladi
11 bagpip
2 bah
3 baha
1 bahahahahahahaha
1 bahama
5 bahamadia
10 bail
43 bailey
1 baili
2 baio
1 baion
4 bairnson
1 baisic
18 bait
2 bajillion
1 bak
1 bakbeat
2 bakc
39 bake
64 baker
1 bakeri
2 bakersfield
1 bakground
6 balad
1 baladi
260 balanc
1 balancethi
1 balboa
1 balck
10 bald
3 balderdash
1 baldfac
1 baldhead
1 baldri
4 bale
1 baledri
1 baley
3 balin
3 balk
245 ball
1822 ballad
1 balladcould
4 balladi
1 balladif
28 balladri
1 balladsand
1 balladsboth
1 balladscon
1 balladsitu
1 balladsnext
1 balladwhi
1 balland
30 ballard
1 ballardi
4 ballaz
1 ballbat
1 

1 bettermakav
1 bettermi
1 betteron
1 betterpost
1 betterr
2 betterthan
1 bettertop
1 betterw
28 betti
1 bettween
1 betwe
1 betweenfumbl
1 betweeninx
2 betwixt
2 beulah
1 beunabl
2 beuti
5 beutiful
1 beutyful
5 bev
3 bevan
1 bevcaus
1 bevel
3 beverag
1 beverley
20 beverli
4 bevi
1 bevita
55 bewar
19 bewild
1 bewilder
1 bewilderbeast
1 bewitch
1 beworth
1 bewtween
91 beyonc
1 beyonce
1 beyoncey
2 beyotch
1 bezzi
7 bezzl
1 bf
2 bfavorit
1 bfl
1 bfor
4 bfp
18 bg
1 bgo
1 bgodsmack
16 bguest
1 bgv
1 bh
2 bhangra
16 bi
17 bia
1 biaci
3 biafra
1 bialik
1 biancaniello
35 bias
2 biatch
23 bibl
12 biblic
2 bice
1 bicentenni
1 bichen
4 bicker
1 bickl
1 bickler
1 bicultur
5 bicycl
1 bicyclist
1 bicyl
27 bid
1 bidder
5 biddi
1 bidni
2 bido
1 bie
15 bieber
5 bien
1 bieng
1 bieni
1 bienvenido
1 bier
1 bifida
2883 big
1 bigcatrecord
19 bigg
7 bigga
158 bigger
533 biggest
556 biggi
1 biggiedead
1 biggiest
1 bigit
1 bigmac
3 bigmouth
3 bigot
2 bigotri
1 bigstandout
3 bigtim
5 bike
6 biker
5 bikini
14 bi

163 brain
1 braincel
2 brainchild
4 braindead
19 brainer
2 brainfreez
4 braini
20 brainless
2 brainlessli
1 brainnn
9 brainstorm
1 brainvil
27 brainwash
7 brake
1 brambl
1 bramlett
150 branch
221 brand
55 brandi
1 brandish
6 brando
46 brandon
3 branford
2 brang
5 branigan
1 brann
1 branson
2 braodi
24 brash
2 brashli
2 brasil
1 brasileiro
1 brasilia
66 brass
1 brassconstruct
15 brassi
38 brat
3 bratti
35 bravado
84 brave
24 braveheart
1 bravehearteddiplomat
1 bravei
1 braver
7 braveri
5 bravest
17 bravo
2 bravura
1 brawl
1 brawler
96 braxton
2 bray
5 brazen
1 brazenli
18 brazil
42 brazilian
6 brazillian
1 bre
7 breach
1 breack
56 bread
2 breadcrumb
1 breadfan
1 breadmanwalk
13 breadth
902 break
4 breakaway
18 breakbeat
3 breakdanc
78 breakdown
13 breaker
1 breakerstringplaypublicityi
1 breakeven
12 breakfast
1 breaki
20 breakin
4 breakingli
8 breakneck
31 breakout
1 breakown
1 breaksbian
3 breakthough
134 breakthrough
3 breakthru
1 breaktweak
79 breakup
1 breakway
1 breal
3 breast
1 br

...

1 tiff
6 tiffani
2 tift
1 tigarah
43 tiger
8 tigermilk
3 tigger
1 tigh
1 tigher
1 tighest
1 tighht
1 tighrop
763 tight
1 tightbottom
17 tighten
31 tighter
28 tightest
1 tightjaw
30 tightli
9 tightrop
1 tightwir
4 tigr
1 tih
2 tiheem
1 tiiiimmmesss
5 tijuana
1 tijuanna
3 tik
1 tikaram
8 tiki
122 til
1 tila
2 tile
166 till
1 tillbrook
28 tillerman
1 tilli
7 tillman
3 tilo
9 tilt
80 tim
1 timabaland
1 timaland
1 timba
1 timbalaland
161 timbaland
1 timbalandhonor
1 timbalnd
2 timber
1 timberblak
1 timberfak
67 timberlak
1 timberlakesqu
16 timberland
1 timberlandfollow
1 timberwolf
14 timbo
27 timbr
9561 time
1 timeaerodynamiccrescendollssometh
1 timebefor
1 timebomb
1 timecheck
1 timeemotionsmi
1 timefollow
4 timefram
1 timeh
1 timei
1 timein
1 timekeep
168 timeless
2 timelessli
1 timeli
3 timelin
1 timelost
1 timem
1 timemachin
1 timemi
1 timepiec
12 timer
1 timerunnin
1 timesi
1 timesin
1 timesinc
1 timespac
2 timesrol
1 timessh
1 timesslepp
1 timeswould
1 timesy
1 timeth
1 timewasn
1 ti

4 unbalanc
47 unbear
1 unbearabley
1 unbearavl
4 unbeat
1 unbeatabilti
3 unbecom
1 unbeknown
2 unbeknownst
1 unbeleiv
2 unbeliav
112 unbeliev
3 unbeliv
9 unbias
1 unbidden
9 unborn
1 unborncontus
1 unbound
36 unbreak
6 unbridl
6 unbroken
1 unc
5 uncal
22 uncanni
3 uncannili
1 uncapt
1 uncaptiv
1 uncaptur
1 uncar
4 uncatchi
1 uncategoriz
1 unceas
3 uncensor
3 unceremoni
11 uncertain
8 uncertainti
8 unchain
3 unchalleng
8 unchang
14 uncharacterist
1 uncharacterst
1 uncharactest
1 uncharicterist
7 unchart
2 uncheck
53 uncl
2 unclassifi
2 unclean
7 unclear
4 unclutt
1 uncomfert
31 uncomfort
4 uncommerci
7 uncommon
1 uncommun
1 uncompar
4 uncompl
1 uncomplet
1 uncomplex
2 uncompress
13 uncompromis
1 unconcern
2 unconci
16 uncondit
1 unconfin
1 unconjur
1 unconnect
4 unconsci
1 unconscion
1 unconsol
2 uncontest
1 uncontriv
12 uncontrol
17 unconvent
2 unconvention
10 unconvinc
5 uncool
1 uncoordin
1 uncorrobor
1 uncount
1 uncouth
5 uncov
22 uncr
11 uncredit
3 uncrit
1 unctuos
2 unctuou
1 uncu

5 vacanc
6 vacant
45 vacat
1 vaccin
1 vaccum
1 vachik
2 vacil
1 vacod
1 vaction
10 vacuou
6 vacuum
2 vader
1 vaga
7 vagabond
1 vagari
2 vagrant
79 vagu
2 vaguest
4 vai
27 vain
3 vaini
3 vainli
8 val
2 vale
5 valedictori
1 valencia
1 valenitn
2 valensi
38 valentin
1 valentineormercuri
20 valentino
1 valentinocan
12 valeri
1 valey
1 valhala
2 valhalla
6 valiant
37 valid
2 valium
4 vallanc
1 valle
56 valley
2 valli
2 valo
2 valparaiso
1 valpariso
1 valpraiso
309 valu
27 valuabl
1 valueabl
1 vamo
16 vamp
1 vampi
56 vampir
1 vampish
1 vampto
394 van
2 vancouv
4 vandal
1 vandebilt
1 vandel
4 vandella
1 vanden
5 vandenberg
1 vander
1 vanderbild
6 vanderbilt
1 vanderlyl
1 vanderquack
5 vandi
52 vandross
1 vaneri
94 vanessa
1 vang
16 vanguard
49 vanilla
8 vanilli
31 vanish
1 vanista
25 vaniti
2 vanlos
1 vanna
4 vannelli
1 vannilla
2 vannucci
1 vanquish
1 vantag
1 vanwyngarden
3 vanzant
39 vapid
1 vapidlead
39 vapor
1 var
2 vares
178 vari
1 variac
1 variado
6 varianc
5 variant
91 variat
1 varico

2 whera
102 wherea
3 whereabout
3 wherebi
1 wherebig
1 wheredisc
2 wherehous
1 whererazorlight
22 wherev
4 whet
2 wheter
1 whetton
4 whew
2919 whi
1 whic
7 whichev
1 whichfound
3 whick
4 whiff
3 whig
1 whigga
1 whiggaz
1 whih
1 whihc
1 whil
1 whileown
1 whilest
57 whilst
34 whim
7 whimper
7 whimsi
21 whimsic
3 whin
128 whine
1 whinefest
3 whinehous
2 whiner
1 whinevil
17 whiney
1 whing
78 whini
1 whinney
2 whinni
86 whip
2 whiplash
2 whippet
1 whipsmart
4 whir
16 whirl
1 whirligig
1 whirlpool
15 whirlwind
3 whisk
1 whisker
21 whiskey
1 whiskeyman
2 whiski
1 whisley
138 whisper
12 whisperi
2 whisperth
3 whispi
1 whist
71 whistl
1 whistler
1 whistlestar
3 whit
7 whitch
1209 white
4 whiteboy
4 whitechocolatespaceegg
1 whitefield
2 whitehead
1 whitehous
1 whitelatoiya
1 whiteley
5 whiter
1 whiterememb
9 whitesnak
1 whitest
2 whitewash
4 whitey
25 whitford
1 whith
1 whitish
4 whitley
2 whitlock
1 whitman
134 whitney
1 whitneysong
3 whittl
6 whiz
1 whizz
1 whjole
61 whoa
4 whoah
1 whoalay
1 

3 zeeki
1 zeena
23 zeitgeist
2 zekey
10 zen
5 zender
1 zeneth
17 zenith
1 zennish
1 zenyatta
39 zep
1 zepelin
2 zepellin
1 zepet
1 zepher
1 zephri
11 zephyr
126 zeppelin
1 zeppelinaerosmithqueenpink
1 zepplen
9 zepplin
1 zeppo
1 zerba
145 zero
1 zerostat
1 zerox
6 zest
1 zesti
2 zeta
1 zeu
7 zevon
1 zey
2 zhane
1 zhit
1 zhou
1 zi
1 ziad
1 zietgeist
3 zietgiest
2 zig
8 ziggi
1 ziggurat
1 zigzag
8 zilch
1 zildjian
5 zilla
12 zillion
1 zillionair
10 zimbabw
1 zimbabwean
1 zimmer
3 zimmerman
1 zimmi
1 zin
1 zine
4 zing
1 zinger
7 zinner
9 zion
1 zionon
1 ziontrain
11 zip
1 zipcod
1 ziplock
3 zipper
3 zippi
1 zirconia
1 zither
2 zito
1 zmedkun
1 znasthat
1 znay
1 zocalo
6 zodiac
18 zoe
1 zoltan
1 zomba
71 zombi
1 zombiefi
1 zombifi
1 zon
83 zone
1 zonk
1 zonoscop
13 zoo
1 zooey
1 zooland
7 zoom
9 zoomer
2 zooropa
1 zoot
2 zorn
8 zowe
1 zoyd
1 zpapa
1 zro
2 zs
1 zthe
1 ztuke
1 zulu
2 zumba
2 zune
1 zup
1 zurich
1 zuton
7 zwan
1 zweig
2 zydeco
1 zygot
22 zz
8 zztop
3 zzz
1 zzzzz
1 zzzzzz
1 zz

In [24]:
X_train, X_test, y_train, y_test  = train_test_split(corpus_data_features_nd[0:len(amazon1)],
                                                     amazon1.Sentiment, test_size=0.3, random_state=42)

In [25]:
from sklearn.linear_model import LogisticRegression
    
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)

In [26]:
y_pred = log_model.predict(X_test)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       -1.0       0.66      0.66      0.66      1747
        0.0       0.58      0.56      0.57      2062
        1.0       0.67      0.70      0.68      1828

avg / total       0.63      0.63      0.63      5637

